In [ ]:
import os
import json

import numpy as np

import matplotlib.pyplot as plt

#from notebook_helpers import *

In [ ]:
class ExperimentResults():
    
    def __init__(self, **kwargs):
        
        self.nan = np.sqrt(-1)
        
        if "filepath" in kwargs.keys():
            
            self.load_results(kwargs["filepath"])
            
    def load_results(self, filepath):
        
        raw_data = np.load(filepath)
        
        self.columns = ['uuid', 'persistence', 'max_persistence',\
                        'bit', 'dt', 'kr', 'dx', 'image_filename']
        
        self.uuid = raw_data[1:, 0]
        self.raw_data = raw_data
        
        self.persistence = np.array(raw_data[1:, 1], dtype=int)
        self.max_persistence = np.array(raw_data[1:, 2], dtype=int)
        self.bit = np.array(raw_data[1:, 3], dtype=int)
        self.dt = np.array(raw_data[1:, 4], dtype=float)
        self.kr = np.array(raw_data[1:, 5], dtype=int)
        
        raw_dx = [elem if elem != "na" else self.nan for elem in raw_data[1:, 6]]
        self.dx = np.array(raw_dx, dtype=float) #raw_data[1:, 7], dtype=float)
        
        
        self.image_filename = np.array(raw_data[1:, 3], dtype=str)

    def __getitem__(self, index):
        
        #import pdb; pdb.set_trace()
        if index[0] == 0:
            return self.uuid[index[1]]
        elif index[0] == 1:
            return self.persistence[index[1]]
        elif index[0] == 2:
            return self.max_persistence[index[1]]
        elif index[0] == 3:
            return self.bit[index[1]]
        elif index[0] == 4:
            return self.dt[index[1]]
        elif index[0] == 5:
            return self.kr[index[1]]
        elif index[0] == 6:
            return self.dx[index[1]]
        elif index[0] == 7:
            return self.image_filename[index[1]]
        
    def __len__(self):
        
        return self.persistence.shape[0]
        
        

In [ ]:
def old_plot_as_planes(exp_data, max_steps=1024,  cmap_name="plasma", fig=None, ax=None):

    cmap_buffer = 0.25
    
    def my_cmap(x):
        
        x = x / np.max(x)
        
        my_cmap = plt.get_cmap(cmap_name)
        
        min_x = cmap_buffer * x 
        modify = 1 - cmap_buffer
        
        return my_cmap(modify * x )
    
    max_kr = np.max(exp_data[:,1])
    
    unique_bits = np.unique(exp_data[:,2])
    unique_bits.sort()

    number_columns = unique_bits.shape[0]

    
    if fig is None:
        fig, ax = plt.subplots(1, number_columns+1, \
                figsize=(number_columns*4, 4), gridspec_kw={"width_ratios": [1] * number_columns + [0.25]})
    
    for ii in range(number_columns):
        plot_data = exp_data[exp_data[:,2] == unique_bits[ii]]
        
        plot_colors = my_cmap(np.array(plot_data[:,3] * (1-cmap_buffer), dtype=np.uint32))
        
        ax[ii].scatter(plot_data[:,0], plot_data[:,1], \
                alpha=0.25, s=7.5,# edgecolor=[0.85,0.85,0.85],\
                c=plot_colors) #, vmin=0, vmax=max_steps * (1+cmap_buffer))

        my_title = f"Persistence with {unique_bits[ii]}-bit precision"
        ax[ii].set_title(my_title)
        ax[ii].set_xlabel("time step dt")
        ax[ii].set_ylabel("kernel radius kr")
        ax[ii].set_xticks(np.arange(0.0, 1.1, 0.1))
        ax[ii].set_yticks(np.arange(0.0, max_kr+int(max_kr/10), int(max_kr/10)))
        
    
    color_grades = 1000 #int(1000 * (1+cmap_buffer))
        
    my_colors =np.arange(0.0, 1.0, 1/color_grades)[:, None] * np.ones((color_grades, 64))
    
    ax[-1].imshow(my_colors, cmap=cmap_name)
    ax[-1].set_yticks(np.arange(0.0, color_grades+color_grades/4., color_grades/4))
    ax[-1].set_yticklabels([int(elem) for elem in np.arange( .0, 1.0+1/4., 1/4.) * max_steps * (1+cmap_buffer)])
    ax[-1].set_xticklabels("")
    ax[-1].set_xlabel("steps")
    plt.tight_layout()
    
    plt.tight_layout()
    
    return fig, ax

def plot_as_planes(exp_data, max_steps=4096, cmap_name="plasma", fig=None, ax=None):
        
    cmap_buffer = 0.25
    
    def my_cmap(x):
        
        x = x / np.max(x)
        
        my_cmap = plt.get_cmap(cmap_name)
        
        min_x = cmap_buffer * x 
        modify = 1 - cmap_buffer
        
        return my_cmap(modify * x )
    
    
    if np.isnan(exp_data[6,1:].mean()):
        print("no kernel r")
        max_kr = np.max(exp_data[5,:])
        min_kr = np.min(exp_data[5,:])
    else:
        max_kr = np.max(1 / exp_data[6,:])
        min_kr = np.min(1 / exp_data[6,:])
    
    unique_bits = np.unique(exp_data[3,:])
    unique_bits.sort()

    number_columns = unique_bits.shape[0]

    if fig is None:
        fig, ax = plt.subplots(1, number_columns+1, \
                figsize=(number_columns*4, 4), gridspec_kw={"width_ratios": [1] * number_columns + [0.25]})

    for ii in range(number_columns):
        
        if np.isnan(exp_data[6,:].mean()):
            
            plot_data_0 = exp_data[4,:][exp_data[3,:] == unique_bits[ii]]
            plot_data_1 = exp_data[5,:][exp_data[3,:] == unique_bits[ii]]
        else:
            plot_data_0 = exp_data[4,:][exp_data[3,:] == unique_bits[ii]]
            plot_data_1 = 1 / exp_data[6,:][exp_data[3,:] == unique_bits[ii]]
        
        plot_colors = my_cmap(np.array(exp_data[1,:][exp_data[3,:] == unique_bits[ii]], dtype=np.uint32))
        ax[ii].scatter(plot_data_0, plot_data_1, \
                alpha=0.425, s=7.5,\
                c=plot_colors)

        my_title = f"Persistence with {unique_bits[ii]}-bit precision"
        ax[ii].set_title(my_title)
        ax[ii].set_xlabel("time step dt")
        
        if np.isnan(exp_data[6,:].mean()):
            ax[ii].set_ylabel("kernel radius")            
            ax[ii].set_yticks(np.arange(min_kr, max_kr+max_kr/10, max_kr/10))
            
        else:
            ax[ii].set_ylabel("1 / dx")
            ax[ii].set_yticks(np.arange(min_kr, max_kr*(1+1/10), max_kr/10))
                
        ax[ii].set_xticks(np.arange(0.0, 1.1, 0.1))
        
        
    color_grades = 1000
    my_colors =np.arange(0.0, 1.0, 1/color_grades)[:, None] * np.ones((color_grades, 64))
    
    ax[-1].imshow(my_colors, cmap=cmap_name)
    ax[-1].set_yticks(np.arange(0.0, color_grades+color_grades/4., color_grades/4))
    ax[-1].set_yticklabels([int(elem) for elem in np.arange( .0, 1.0+1/4., 1/4.) * max_steps * (1+cmap_buffer)])
    ax[-1].set_xticklabels("")
    ax[-1].set_xlabel("steps")
    plt.tight_layout()
    
    return fig, ax
    
    
def visualize_all_results(results_path, exp_idx=None, use_old=False): 
    """
    For quickly visualizing all results in a folder
    """
    
    temp_results_dir = os.listdir(results_path)
    
    # if a specific results_path is given, skip this part
    exp_folder = None

    results_dir = []
    for item  in temp_results_dir:
        if item.startswith("exp"):
            results_dir.append(item)

    results_dir.sort()

    for idx, folder in enumerate(results_dir):
        print(f"{idx}      {folder}")

    for idx, folder_name in enumerate(results_dir):

        print(f"index {idx}, folder name {folder_name}")

        exp_folder = os.path.join(results_path, folder_name)
        if os.path.isdir(exp_folder):
            metadata_path = os.path.join(exp_folder, "metadata.json")
            if os.path.exists(metadata_path):
                my_dir_list = os.listdir(exp_folder)
                for filename in my_dir_list:
                    if filename.startswith("results"):
                        experiment_results_path = os.path.join(exp_folder, filename)
                        my_filename = filename
                        if use_old:
                            my_data = np.load(experiment_results_path)
                        else:
                            my_data = ExperimentResults(filepath=experiment_results_path)

                with open(metadata_path, "r") as f:
                    metadata = json.load(f)

                if use_old:
                    fig, ax = old_plot_as_planes(my_data, \
                        max_steps=1+np.max(my_data[:,3]))
                else:
                    fig, ax = plot_as_planes(my_data, \
                        max_steps=1+np.max(my_data[1,:]))

                fig.suptitle(f"{exp_folder}\n{my_filename[-64:]}", fontsize=22)
                plt.tight_layout()
                plt.show()
                

def visualize_results(results_path, exp_idx=None, use_old=False, fig=None, ax=None):
    """
    For visualizing specific results, returning the fig, ax for downstream work
    """
    
    
    temp_results_dir = os.listdir(results_path)
    
    if exp_idx is not None:
        if "metadata.json" in temp_results_dir:
            exp_folder = results_path
        else:
            results_dir = []
            for item  in temp_results_dir:
                if item.startswith("exp"):
                    results_dir.append(item)
            
            results_dir.sort()
            exp_folder = os.path.join(results_path, results_dir[exp_idx])

    else:
        results_dir = []
        for item  in temp_results_dir:
            if item.startswith("exp"):
                results_dir.append(item)

        results_dir.sort()
        
        for idx, folder in enumerate(results_dir):
            print(f"{idx}      {folder}")

        exp_idx = input(f"please choose an experiment folder (by index number) to visualize. (-1 to visualize all).")
        exp_idx = int(exp_idx)
        exp_folder = os.path.join(results_path, results_dir[exp_idx])
       
    exp_dir = os.listdir(exp_folder)
    metadata_path = os.path.join(exp_folder, "metadata.json")

    for filename in exp_dir:
        if filename.startswith("results"):
            experiment_results_path = os.path.join(exp_folder, filename)
            my_filename = filename

    if use_old:
        my_data = np.load(experiment_results_path)
    else:
        my_data = ExperimentResults(filepath=experiment_results_path)

    with open(metadata_path, "r") as f:
        metadata = json.load(f)
        
    
    if fig is None:
        if use_old:
            
            max_steps = 1+np.max(my_data[:,3])
            fig, ax = old_plot_as_planes(my_data, max_steps=max_steps)
        else:
            max_steps = 1+np.max(my_data[1,:])
            fig, ax = plot_as_planes(my_data, max_steps=max_steps)
    else:
        if use_old:
            max_steps = 1+np.max(my_data[:,3])
            fig, ax = old_plot_as_planes(my_data, fig=fig, ax=ax, max_steps=max_steps)
        else:
            max_steps = 1+np.max(my_data[1,:])
            fig, ax = plot_as_planes(my_data, fig=fig, ax=ax, max_steps=max_steps)

    fig.suptitle(f"{exp_folder}\n{my_filename[-64:]}", fontsize=22)
    plt.tight_layout()
    
    number_samples = len(my_data)
    
    print(f" {my_filename}")
    
    return fig, ax, number_samples, max_steps



In [ ]:
# set to True to save figures to disk
save_figs = False

In [ ]:
default_results_path = os.path.join("..", "results")

In [ ]:
# experiments with max_steps=1024. 
early_results_path = os.path.join("..", "results", "exp_06a")
"""
exp_06a key
0      exp_1687632498404 results_neurosingle_glider000.npy
1      exp_1687646901040 results_neurosynorbium000.npy
2      exp_1687661317458 results_neuroo_bicaudatus_ignis000.npy
3      exp_1687675773160 results_exp_local_glaberish_geminium_simple_halt_1649264299_seed11_config_evolved_ca_slow_glider000.npy
4      exp_1687690176820 results_exp_local_glaberish_geminium_simple_halt_1649264299_seed7_config_spiky_frog_egg000.npy
5      exp_1687704602495 results_s613_s613_frog000.npy
6      exp_1687719006176 results_s643_s643_frog000.npy
7      exp_1687733414886 results_smoothlife_single_glider000.npy
8      exp_1687747821513 results_scutium_gravidus_single000.npy
9      exp_1687762228846 results_geminium_hydrogeminium_natans000.npy
10      exp_1687776647843 results_orbium_orbium000.npy
11      exp_1687791085602 results_synorbium_orbium000.npy
12      exp_1687805485748 results_triscutium_solidus_triscutium000.npy
13      exp_1688008554524 results_gol_reflex_glider000.npy
14      exp_1688015754887 results_gol_small_spaceship000.npy
15      exp_1688098875785 results_neurorbium000.npy
16      exp_1688136149309 results_neuroreflex_glider000.npy
17      exp_1688604434229 results_s613_fast_wobble_glider000.npy
18      exp_1688611645783 results_s643_s643_3wide_wobble_glider000.npy
19      exp_1688618846573 results_geminium2_wobble_glider000.npy
20      exp_1688626056754 results_neurowobble_glider000.npy

not used in the paper:
4 exp_1687690176820  (not a glider),
13 exp_1688008554524 (not continuous CA), 
14 exp_1688015754887 (not continuous CA),
16 exp_1688136149309 (not continuous CA) 
"""

# more results, with more informative experiment names and more comprehensive metadata
results_path = os.path.join("..", "results", "exp_06b")

# Additional results, with 4096 steps
additional_results_path = os.path.join("..", "results", "exp_06c")

In [ ]:
## neurowobble glider

# 1024 step experiment
early_results_path = os.path.join("..", "results", "exp_06a")
fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=20)
fig.suptitle(f"Neurowobble NCA glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)
print(f"max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/neurowobble_glider_1k.png")
plt.show()

# experiment with 4096 steps
results_path = os.path.join("..", "results", "exp_06b")
fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=7)
total_samples = number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Neurowobble glider 4k stability \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/neurowobble_glider_4k.png")
plt.show()


In [ ]:
## neurosynorbium

early_results_path = os.path.join("..", "results", "exp_06a")
fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=1)
fig.suptitle(f"Neurosynorbium NCA glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)
print(f"max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/neurosynorbium_glider_1k.png")
plt.show()

results_path = os.path.join("..", "results", "exp_06b")
fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=13)
total_samples = number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Results for neurosynorbium glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/neurosynorbium_glider_4k.png")
plt.show()


In [ ]:
## neurorbium

early_results_path = os.path.join("..", "results", "exp_06a")
results_path = os.path.join("..", "results", "exp_06b")
additional_results_path = os.path.join("..", "results", "exp_06c")

fig, ax, number_samples, max_steps = visualize_results(early_results_path, \
        use_old=True, exp_idx=15)

fig.suptitle(f"Neurorbium NCA glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)
print(f"max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/neurorbium_1k.png")
plt.show()


fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False,\
        exp_idx=11)
total_samples = number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Neurorbium glider 4k stability \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/neurorbium_4k.png")
plt.show()


In [ ]:
## neurosingle

early_results_path = os.path.join("..", "results", "exp_06a")
results_path = os.path.join("..", "results", "exp_06b")
additional_results_path = os.path.join("..", "results", "exp_06c")

fig, ax, number_samples, max_steps = visualize_results(early_results_path, \
        use_old=True, exp_idx=0)

fig.suptitle(f"Neurosingle NCA glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)
print(f"max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/neurosingle_1k.png")
plt.show()


fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False,\
        exp_idx=3)
total_samples = number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Results for neurosingle glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/neurosingle_4k.png")
plt.show()


In [ ]:
## s11 slow glider 


fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=3)
fig.suptitle(f"s11 slow glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f"max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/s11_slow_glider_1k.png")
plt.show()

total_samples = 0

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=0)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=15)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Results for s11 slow glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)


if save_figs:
    plt.savefig("../assets/s11_slow_glider_4k.png")
plt.show()


In [ ]:
## s643 wobbler

fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=18)
fig.suptitle(f"s643 wobble glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s643_wobbler_1k.png")
plt.show()

total_samples = 0

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=15)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=7)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"Results for  s643 wobble glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/s643_wobbler_4k.png")
plt.show()


In [ ]:
## s643 frog

fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=10)
fig.suptitle(f"s643 frog glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s643_frog_1k.png")
plt.show()

total_samples = 0

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=1)
total_samples = number_samples

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, fig=fig, ax=ax, exp_idx=5)
total_samples = number_samples

fig.suptitle(f"Results for  s643 frog glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s643_frog_4k.png")
plt.show()

In [ ]:
## s613 wobbler

fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=17)
fig.suptitle(f"s613 wobble glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s613_wobbler_1k.png")
plt.show()

total_samples = 0

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=4)
total_samples += number_samples

print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=3)
total_samples += number_samples

fig.suptitle(f"Results for s613 wobble glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)
print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s613_wobble_4k.png")
plt.show()

In [ ]:
## s613 frog

fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=5)
fig.suptitle(f"s613 frog glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f"max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/s613_frog_1k.png")
plt.show()

total_samples = 0

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=5)
total_samples += number_samples

print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=2)
total_samples += number_samples

print(f"max steps: {max_steps}")

fig.suptitle(f"Results for s613 frog glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/s613_frog_4k.png")
plt.show()


In [ ]:
##### Hydrogeminium natans wobbler


fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=19)
fig.suptitle("${\it H.}$" + " " + "${\it natans}$" + " wobble glider " + f"discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f" max steps: {max_steps}")


if save_figs:
    plt.savefig("../assets/h_natans_wobbler_1k.png")

fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=10)
total_samples = number_samples

print(f" max steps: {max_steps}")

fig.suptitle(f"Results for " + "${\it H.}$" + " " + "${\it natans}$" + " wobble glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/h_natans_wobbler_4k.png")
plt.show()



In [ ]:
## Hydrogeminium natans cucumber

fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=9)
fig.suptitle("${\it Hydrogeminium}$" + " " + "${\it natans}$" + " glider " + f"discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f" max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/h_natans_cucumber_1k.png")


fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=14)
total_samples = number_samples

print(f" max steps: {max_steps}")
fig.suptitle(f"Results for " + "${\it Hydrogeminium}$" + " " + "${\it natans}$"+ " glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

print(f" max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/h_natans_cucumber_4k.png")
plt.show()

In [ ]:
## Triscutium 


fig, ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=12)
fig.suptitle("${\it Triscutium}$" + " " + "${\it solidus}$" + " glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f" max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/triscutium_solidus_1k.png")


fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=19)
total_samples = number_samples

print(f" max steps: {max_steps}")
fig.suptitle(f"Pooled results for " + "${\it Triscutium}$" + " " + "${\it solidus}$" + " glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/triscutium_4k.png")
plt.show()



In [ ]:
## Scutium gravidus


pre_fig, pre_ax, number_samples, max_steps = visualize_results(early_results_path, use_old=True, exp_idx=8)
pre_fig.suptitle("${\it Scutium}$" +" "+ "${\it gravidus}$" + " glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f" max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/scutium_gravidus_1k.png")


fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False,\
        exp_idx=8)
total_samples = number_samples

print(f" max steps: {max_steps}")
fig.suptitle(f"4k results for " + "${\it Scutium}$" +" "+ "${\it gravidus}$" + " glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)


if save_figs:
    plt.savefig("../assets/scutium_gravidus_4k.png")
plt.show()



In [ ]:
## Orbium orbium

fig, ax, number_samples, max_steps = visualize_results(early_results_path, exp_idx=10, use_old=True)
fig.suptitle("${\it Orbium}$" + " glider discretization stability \n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)
print(f" max steps: {max_steps}")

if save_figs:
    plt.savefig("../assets/orbium_1k.png")
plt.show()

fig, ax, number_samples, max_steps = visualize_results(results_path, exp_idx=6, use_old=False)
total_samples = number_samples

print(f" max steps: {max_steps}")
fig.suptitle(f"4k Results for " + "${\it Orbium}$" + " glider \n"\
             f"max steps = {max_steps}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/orbium_4k.png")
plt.show()



# SmoothLife 

## Results for the SmoothLife single glider

In [ ]:
## Pool SmoothLife single glider results 

fig, ax, number_samples, max_steps = visualize_results(early_results_path, exp_idx=7, use_old=True)

fig.suptitle(f"SmoothLife single glider discretization stability\n"
             f"max steps = {max_steps}, n={number_samples}", fontsize=28)

print(f" max steps: {max_steps}")
if save_figs:
    plt.savefig("../assets/smoothlife_glider_1k.png")
plt.show()

fig, ax, number_samples, max_steps = visualize_results(results_path, exp_idx=16, use_old=False)
total_samples = number_samples
print(f" max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, exp_idx=12, use_old=False, fig=fig, ax=ax)
total_samples += number_samples

print(f" max steps: {max_steps}")
fig.suptitle(f"Pooled results for SmoothLife single glider \n"\
             f"max steps = {4096}, n={total_samples}", fontsize=28)
if save_figs:
    plt.savefig("../assets/smoothlife_4k.png")
plt.show()



In [ ]:
###### u-skate berrycup
# Note that the scheme I used to detect glider persistence for U-Skate World,
# (correlation with the original glider pattern), is unreliable, 
# as revealed by auditing many images of final grid states.
# Therefore the persistence scatter plots for U-Skate World here are approximate at best, and may be misleading. 

total_samples = 0

results_path = os.path.join("..", "results","exp_06b")
fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=18)
total_samples += number_samples
print(f"max steps: {max_steps}")


additional_results_path = os.path.join("..", "results", "exp_06c")
fig, ax, number_samples, max_steps = visualize_results(additional_results_path, \
        use_old=False, fig=fig, ax=ax, exp_idx=1)
total_samples += number_samples
print(f"max steps: {max_steps}")



fig.suptitle(f"U-Skate BerryCup stability\n"\
             f"max steps={131072}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/berrycup_131k.png")
plt.show()

In [ ]:
## u-skate glider 

total_samples = 0

results_path = os.path.join("..", "results", "exp_06b")
fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=9)
total_samples += number_samples
print(f"max steps: {max_steps}")

additional_results_path = os.path.join("..", "results", "exp_06c")
fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=6)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=13)
total_samples += number_samples
print(f"max steps: {max_steps}")                     


fig.suptitle(f"U-Skate 'u' glider stability \n"\
             f"max steps={131072}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/uskate_glider_131k.png")
plt.show()


In [ ]:
# 17, 10
# 26, 1
# ## u-skate daedalus 


results_path = os.path.join("..", "results", "exp_06b")
fig, ax, number_samples, max_steps = visualize_results(results_path, use_old=False, exp_idx=12)
total_samples = number_samples

print(f"max steps: {max_steps}")

additional_results_path = os.path.join("..", "results", "exp_06c")


fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=10)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig, ax, number_samples, max_steps = visualize_results(additional_results_path, use_old=False, \
        fig=fig, ax=ax, exp_idx=11)
total_samples += number_samples
print(f"max steps: {max_steps}")

fig.suptitle(f"U-Skate Daedalus stability\n"\
             f"max steps={131072}, n={total_samples}", fontsize=28)

if save_figs:
    plt.savefig("../assets/daedalus_131k.png")
plt.show()


In [ ]:
# You can visualize all results in a directory as scatter plots, 
# to get a summary view of the results of an experiment

_ = visualize_all_results(os.path.join(default_results_path, "exp_06c"))
plt.show()

In [ ]:
# choose experiment from a printed list
# must use 'use_old=True' for exp_06a

_ = visualize_results(os.path.join(default_results_path, "exp_06a"), use_old=True)
plt.show()

In [ ]:
# choose experiment from a printed list

_ = visualize_results(os.path.join(default_results_path, "exp_06c"))
plt.show()

In [ ]:
# visualize the results found in a specific exp directory

specific_folder = os.path.join(default_results_path, "exp_06c",\
        "exp_8444c4cc_8229_493d_af2b_11e26036784b_CCA_s643_s643_frog000")

pattern_name = os.path.split(specific_folder)[-1][-12:-3]
fig, ax, number_samples, max_steps  = visualize_results(specific_folder, 0)

# figure can be modified before display
plt.suptitle(f"{pattern_name} persistence\n{number_samples} samples, max steps ={max_steps}.", fontsize=24)
plt.tight_layout()
plt.show()